<a href="https://colab.research.google.com/github/byambaa1982/mongol_tweets/blob/master/tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

### Aшигласан сангууд

In [3]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import pandas as pd
import numpy as np
import regex as re
import os
import json
import time
import sys
import io
from datetime import datetime, date, time, timedelta
import pandas_gbq


# Google Cloud Platform - Using Machine Learning APIs  ).

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [ ]:
# import getpass

# APIKEY = getpass.getpass()

 ### Access google driver using google credentials

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
os.chdir('/content/gdrive/My Drive/fiverr/tweets')
with open("twitter_credentials.json", "r") as file:
  creds = json.load(file)
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
api = tweepy.API(auth, wait_on_rate_limit=True)
os.chdir('/content/gdrive/My Drive/fiverr/byamba/data')

### Functions - функцүүд

In [8]:
def desc_users(account_list):
	users={}
	users["name"]=[]
	users["screen_name"]=[]
	users["description"]=[]
	users["statuses_count"]=[]
	users["friends_count"]=[]
	users["followers_count"]=[]
	users["account_age"]=[]
	users["average_tweets_per_day"]=[]
	if len(account_list) > 0:
		for target in account_list:
			try:
				item = api.get_user(target)
				users["name"].append(item.name)
				users["screen_name"].append(item.screen_name)
				users["description"].append(item.description)
				users["statuses_count"].append(item.statuses_count)
				users["friends_count"].append(item.friends_count)
				users["followers_count"].append(item.followers_count)
				tweets = item.statuses_count
				account_created_date = item.created_at
				delta = datetime.utcnow() - account_created_date
				account_age_days = delta.days
				users["account_age"].append(account_age_days)
				# print("Account age (in days): " + str(account_age_days))
				if account_age_days > 0:
					# print("Average tweets per day: " + "%.2f"%(float(tweets)/float(account_age_days)))
					users["average_tweets_per_day"].append(float(tweets)/float(account_age_days))
			except:
				print("not found:{}".format(target))
	df=pd.DataFrame(users, columns=users.keys())
	return df


In [9]:
def get_tweets_csv(search_words, date_since):
  new_search = search_words + " -filter:retweets"

  tweets = tweepy.Cursor(api.search,
                              tweet_mode="extended", 
                              q=new_search,
                              lang="ru",
                              since=date_since).items(2000)

  users_locs = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.user.location
                ,tweet.favorite_count,tweet.retweet_count] for tweet in tweets]
  users_locs
  # Checks if there are coordinates attached to tweets, if so extracts them
  # users_locs['Tweet Coordinates'] = users_locs.apply(extract_coordinates,axis=1)

  tweet_text = pd.DataFrame(data=users_locs, 
                      columns=["id", "user", "tweets","date","location","favorites","retweets"])

  # tweet_text['Tweet Coordinates'] = tweet_text.apply(extract_coordinates,axis=1)
  # tweet_text.to_csv(search_words+".csv", index=False)
  return tweet_text

In [10]:
def get_tweets_by_location(api, latitude, longitude, radius, search_words, date_since):
    new_search = search_words + " -filter:retweets"
    tweets = tweepy.Cursor(api.search,
                           q=new_search,
                           tweet_mode="extended", 
                           geocode=f"{latitude},{longitude},{radius}km",
                           lang="ru",
                           since=date_since).items(2000)
    users_locs = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.user.location
                ,tweet.favorite_count,tweet.retweet_count] for tweet in tweets]
    tweet_text = pd.DataFrame(data=users_locs, 
                      columns=["id", "user", "tweets","date","location","favorites","retweets"])
    return tweet_text

In [11]:
def get_user_stats(api, user_id):
    user = api.get_user(user_id)
    return {
        "user_name":user_id,
        "followers_count": user.followers_count,
        "friends_count": user.friends_count,
        "listed_count": user.listed_count,
        "favourites_count": user.favourites_count,
        "statuses_count": user.statuses_count
    }

In [28]:
def print_ids(df, col, num):
  for name in df[str(col)].to_list()[0:num]:
    print('@'+name+',')

In [13]:
def get_name(tweets, df):
  tweets=tweets[['user', 'tweets', 'favorites','date']]
  tweets.columns=pd.Series(['screen_name', 'tweets', 'favorites','date'])
  temp=df[['name', 'screen_name']]
  tweets=pd.merge(tweets, temp, how='left', on=['screen_name'] )
  tweets=tweets[['name', 'screen_name','tweets', 'favorites','date']]
  return tweets

In [14]:
def get_stats_with_name(tweets):
  user_name_list=[name for name in tweets.user.to_list()]
  df=desc_users(user_name_list)
  df['account_age_year']=df.account_age.map(lambda x: x/360)
  df.average_tweets_per_day=df.average_tweets_per_day.map(lambda x: round(x,1))
  df.account_age_year=df.account_age_year.map(lambda x: round(x,1))
  df=df[['name', 'screen_name', 'statuses_count','followers_count', 'average_tweets_per_day','account_age_year']]
  return df

In [15]:
def display_full(tweets):
# Set the display option to show the full text
  pd.set_option('display.max_colwidth', None)

  display(tweets)

In [16]:
# Get Followers ids
username = "enkhbatbyambats"  # Replace with the username you want to get the followers for
user = api.get_user(username)
user_id = user.id_str

# Get the user IDs for all the followers of the given user
follower_ids = []
for follower_id in tweepy.Cursor(api.followers_ids, user_id=user_id).items():
    follower_ids.append(follower_id)

# Print the follower IDs
# print(follower_ids)


In [17]:
# Get the user ID for the given Twitter username
username = "enkhbatbyambats"  # Replace with the username you want to get the followees for
user = api.get_user(username)
user_id = user.id_str

# Get the user IDs for all the accounts that the given user is following
followee_ids = []
for followee_id in tweepy.Cursor(api.friends_ids, user_id=user_id).items():
    followee_ids.append(followee_id)

# Print the followee IDs
print(followee_ids)

[132568511, 1537609540803641344, 1477847745348370432, 19067150, 1081012147239428096, 1528782657735208961, 1158195831201308673, 1527666073914511363, 1335132884278108161, 1271899909675859968, 4416456732, 1075142947270742016, 1488144856631451655, 1349982942873878528, 524205406, 3831370639, 625761010, 933498545629315072, 1192931493414363139, 630995607, 1120633726478823425, 24949956, 861945291926233088, 1226943756, 219720195, 1128525318740779008, 319988168, 1436666545506185225, 1290494413362233345, 319131439, 1384908454146445316, 1443631963454181376, 1470351572295110656, 1327266713054941184, 1126512593579823109, 1498176452084256772, 1891490382, 524222372, 321835721, 839488183813337090, 1387552712, 6643802, 391852521, 2668655310, 209037902, 54819349, 62448898, 80777173, 1366385154332188675, 72031429, 277530827, 622518593, 454540822, 1922719790, 957152929, 703406372, 394082002, 1338574002, 125965499, 236490748, 1537796658, 2168185345, 1006419421244678144, 403842551, 62980476, 9047005299888209

## Pull Data

In [6]:
DATE="2023-02-25"

### Дагагчидынхаа дата г сонирхсон нь

In [ ]:
my_followers=desc_users(follower_ids)
my_followers=my_followers.sort_values("followers_count", ascending=False)
display(my_followers)

,name,screen_name,description,statuses_count,friends_count,followers_count,account_age,average_tweets_per_day
262,"MarQuis Trill | Crypto, NFT & Blockchain Analyst",6BillionPeople,"Cryptocurrency + NFT + Blockchain Educator, In...",183345,4046930,2049144,5053,36.284386
322,Entrepreneurs,TheSuccessKing,"I Quote & Share Articles About Entrepreneurs, ...",46684,329616,404801,4272,10.927903
646,Oyungerel Цэдэвдамба,oyunlt,Human rights activist & author of The Green Ey...,68376,25006,299634,4996,13.686149
543,Mongol Tsakhia ELBEGDORJ,elbegdorj,"President 2009-2017, PM 1998,2004 of Mongolia,...",4842,7312,265756,4419,1.095723
471,Баабар,baabar,Баабар,670,46628,173293,5030,0.133201
...,...,...,...,...,...,...,...,...
253,oyunaa,oyunaa19877848,,3,9,1,1761,0.001704
633,onon,onon16973338,,0,20,1,3644,0.000000
545,Baaska,Baaska888,,0,9,1,3577,0.000000
55,fibonaccistress,nkhjn0411,,56,523,0,2192,0.025547


In [ ]:
df=my_followers
df['account_age_year']=df.account_age.map(lambda x: x/360)
df.average_tweets_per_day=df.average_tweets_per_day.map(lambda x: round(x,1))
df.account_age_year=df.account_age_year.map(lambda x: round(x,1))
df=df[['name', 'screen_name', 'statuses_count','followers_count', 'average_tweets_per_day','account_age_year']]
df.head()

,name,screen_name,statuses_count,followers_count,average_tweets_per_day,account_age_year
262,"MarQuis Trill | Crypto, NFT & Blockchain Analyst",6BillionPeople,183345,2049144,36.3,14.0
322,Entrepreneurs,TheSuccessKing,46684,404801,10.9,11.9
646,Oyungerel Цэдэвдамба,oyunlt,68376,299634,13.7,13.9
543,Mongol Tsakhia ELBEGDORJ,elbegdorj,4842,265756,1.1,12.3
471,Баабар,baabar,670,173293,0.1,14.0


In [ ]:
df.to_csv(f'my_followers_{DATE}.csv',index=False)

### Монголын топ 20 жиргээчийг сонирхсон нь

In [ ]:
# my_followees=desc_users(followee_ids)
# my_followees=my_followees.sort_values("followers_count", ascending=False)
df = my_followees
df['account_age_year'] = df.account_age.map(lambda x: x/360)
df.average_tweets_per_day = df.average_tweets_per_day.map(lambda x: round(x,1))
df.account_age_year = df.account_age_year.map(lambda x: round(x,1))
df = df[['name', 'screen_name', 'statuses_count','followers_count', 'average_tweets_per_day','account_age_year', 'description']]
df_sorted = df.sort_values("followers_count", ascending=False)

In [ ]:
df.to_csv(f'my_followees_{DATE}.csv',index=False)

In [ ]:
# Reset the index of the sorted DataFrame
df_sorted = df_sorted.reset_index(drop=True)
mongol_followees = df_sorted.iloc[64:,:]

In [ ]:
# Reset the index of the sorted DataFrame
df_sorted = df_sorted.reset_index(drop=True)
mongol_followees = df_sorted.iloc[64:,:]
mongol_followees = mongol_followees.drop([65, 66, 67,68, 70, 72, 74, 75, 76, 77, 78, 82, 83, 86, 87, 89, 91, 95])
mongol_followees=mongol_followees.reset_index(drop=True)
mongol_followees.head(20)

,name,screen_name,statuses_count,followers_count,average_tweets_per_day,account_age_year,description
0,Asashoryu第68代横綱朝青龍🇯🇵🇲🇳,Asashoryu,61281,345471,14.8,11.5,第68代横綱 朝青龍 明徳 АсашёорюүАкинори Asashōryū Akino...
1,Battulga Khaltmaa,BattulgaKh,14181,304112,3.3,12.1,Монгол Улсын Ерөнхийлөгч | President of Mongol...
2,Oyungerel Цэдэвдамба,oyunlt,68376,299634,13.7,13.9,Human rights activist & author of The Green Ey...
3,Jargal Dambadarjaa,jargal_defacto,25280,279832,5.4,13.1,"ED Defacto Institute, Editor The Defacto Gazet..."
4,Mongol Tsakhia ELBEGDORJ,elbegdorj,4842,265756,1.1,12.3,"President 2009-2017, PM 1998,2004 of Mongolia,..."
5,Baabar,Baabarniitlelch,36049,264775,9.0,11.1,Vitam impendere vero
6,Enkhbat Dangaasuren,enkhbat,34832,255389,6.8,14.3,"System Engineer/PhD, Mongolian Internet Pionee..."
7,Zandaakhuu Enkhbold,enkhboldz,31233,238296,6.2,14.1,https://t.co/dQUeKWmfCt
8,Temuujin Kh,Temuujin_Kh,38229,237012,7.6,13.9,
9,Ganhuyag Ch. Hutagt,ganhuyag,105679,228843,21.6,13.6,"CEO, Ard Holdings. Co-Founder: Investor Nation..."


In [ ]:
mongol_followees.to_csv(f'my_mongol_followees_{DATE}.csv', index=False)

In [ ]:
df.to_csv(f'my_mongol_followees_{DATE}.csv', index=False)

In [ ]:
df = pd.read_csv(f'my_mongol_followees_{DATE}.csv')

In [ ]:
# df = df.drop([26])
# df = df.reindex()
df=df.head(20)

In [ ]:
df = df[['name', 'screen_name',  'followers_count','average_tweets_per_day', 'account_age_year']]

In [ ]:
df.to_csv(f'top_20_account_{DATE}.csv', index = False)

In [ ]:
active=df.sort_values("average_tweets_per_day", ascending=False)
active.head(20)

,index,name,screen_name,statuses_count,followers_count,average_tweets_per_day,account_age_year,description
17,26,𝓣𝓼.𝓑𝓪𝓽-𝓔𝓻𝓭𝓮𝓷𝓮 🕉,tsbat_IT,612810,151171,148.5,11.5,"Алдаатай, оноотой жирийн нэгэн, Инженер, Фото ..."
25,36,Uyanga Tsoggerel,Uyanga_ts,485392,119888,97.3,13.9,♣🏠 Clubhousehttps://clubhousedb.com/user/uyang...
40,51,Л. ЦАНДЭЛЭГ,Tsandeleg_L,304891,91330,81.0,10.5,Монгол Улсын Тод Манлай Уяач Адуу Тариа хоёрто...
147,158,Nuclear Ganaa,ganaa23mn,256913,30661,68.3,10.4,"Өнгийг биширч битгий хар, гэрэл гэж бодоод сур..."
178,189,БУЯНАА.МН,erdene457,244476,24603,64.4,10.5,"хүний эрх, эрх чөлөө, ардчилал, шударга ёс, па..."
152,163,Амьдралын Санхүү,FinlifeMN,219181,30058,59.4,10.2,Санхүүгийн боловсрол түгээж Хүртээмжтэй хөгжли...
213,224,Batbayar Ochirbat,0Baagii,220636,19583,54.6,11.2,"Think Globally Work Locally , Alumni of London..."
313,324,Uuganbayar Batsaihan,uuganaa_v2,129332,11170,52.2,6.9,"“Those who deny freedom to others, deserve it ..."
112,123,АРИ,Ariunq,70895,39536,50.9,3.9,Жиргээгээ устгаад өг гэж битгий гуйгаад байгаа...
337,348,𝑵.𝑩.𝑩𝒂𝒕𝒔𝒂𝒊𝒌𝒉𝒂𝒏,NBBatsaikhan,176316,9700,50.4,9.7,♏☸


In [ ]:
active.to_csv(f'active_20_{DATE}.csv', index = False)

### Search by location

In [20]:

# DATE="2023-02-25"
# SEARCH_WORD = "*"
# tweets = get_tweets_by_location(api, 47.9186, 106.9175, 10, SEARCH_WORD, DATE)
# tweets = tweets.sort_values("favorites", ascending=False)
df = tweets [['user', 'tweets', 'favorites']]
df.head(10)

,user,tweets,favorites
1398,elbegdorj,Өчигдөр Украйнаар бүтэн өдөр амьсгалав. Хамгий...,232
1564,g_ganbat,"Асгатын давааны хяраас Их Соёны нуруу хүртэлх,...",168
1674,Gumpaa,Тэхдээ Аялалын сайд улсынхаа бараг хамгийн цэв...,162
1944,tsogt_tsogt,Энэ 2 хөшөөг ямар азаар буулгасын. Одоо байга...,160
1576,Gumpaa,Нэгэнт сэдэв болсон дээр нь хэлхэд хар залууга...,133
1868,g_ganbat,"Хүрэлсүх, Оюун-Эрдэнэ нарын бидний хувь эзэмшд...",125
1141,hono_hongoroo,"💡 Дорноговь, Сайншанд хот - миний сайхан нутаг...",106
1110,g_ganbat,Тайлбаргүй. https://t.co/w6097dY5tb,78
1015,sakuna_mn,Ийм юмнууд ТҮК-ийн ажилчдаа авч өгөх юмсан. ht...,67
1625,rdavaadorj,Руслан Хасбулатов гэгч эрхэм миний дисертацийн...,67


In [29]:
print_ids(df, 'user', 10)

@elbegdorj,
@g_ganbat,
@Gumpaa,
@tsogt_tsogt,
@Gumpaa,
@g_ganbat,
@hono_hongoroo,
@g_ganbat,
@sakuna_mn,
@rdavaadorj,


In [21]:
tweets.to_csv(f'search_by_location_{DATE}.csv',index=False)

### Create a chart

In [ ]:

DATE="2023-02-20"
SEARCH_WORD="данх"
negative = get_tweets_csv(SEARCH_WORD, DATE)
negative=negative.sort_values("favorites", ascending=False)
print(f"Total of tweets: {negative.shape[0]}, Total of favorites: {negative.favorites.sum()}, a total of retweets: {negative.retweets.sum()}")
negative.head(10)

Total of tweets: 59, Total of favorites: 157, a total of retweets: 61


,id,user,tweets,date,location,favorites,retweets
43,1627502789877432320,Odbayar1206,Мөнхжин нарын залуусын хэргийг шүгэлддэг данх ...,2023-02-20 02:57:57,Mongolia,35,21
40,1627504729520500736,chinbat84,Шүгэлддэг данх \n• Зохиолтой\n• Тавилтай\n• Мо...,2023-02-20 03:05:39,Mongolia,35,21
42,1627503314740060160,Uyanga_ts,Энэ Данх одоо юун гонгиноон хийцийн бэ? Өөрийг...,2023-02-20 03:00:02,"Ulaanbaatar, Mongolia",23,2
56,1627474828751142915,aagiiah,Айлд ороод хиам идсэн чинь тэр нь #ДАНХ-ыг дол...,2023-02-20 01:06:50,,12,5
50,1627490303044358144,altancom7969,Өчигдөр орой айл бүрийн цэнхэр дэлгэцээр хоосо...,2023-02-20 02:08:20,Улаанбаатар,11,2
35,1627514946358575105,sumiyamgl,"Та нар данх гээд бдаг билү, би хувьдаа @Oyuner...",2023-02-20 03:46:15,Mongolia,8,4
29,1627539938123345922,gan_9999,"Манай Монголд ямар ч адгийн шаарыг,ямар ч чадв...",2023-02-20 05:25:34,,8,3
55,1627476917099143169,ayvengoroe,Шаампар данх https://t.co/CfNEzvLEkB,2023-02-20 01:15:08,"Ulaanbaatar, Mongolia",4,0
15,1627610685114040320,ganzorigt,@UBn_mn @OyunerdeneMN @elbegdorj @Enkhbayar_ba...,2023-02-20 10:06:41,Mongolia,3,0
25,1627558413952757760,OilooSM,@Babrosk Данх зүлгээд л үү? Сүүлдээ нөгөөх нь ...,2023-02-20 06:38:59,Ulaanbaatar,2,0


In [ ]:
neg_user_stats=get_stats_with_name(negative.head())
neg_user_stats.sort_values("followers_count", ascending=False)

,name,screen_name,statuses_count,followers_count,average_tweets_per_day,account_age_year
2,Uyanga Tsoggerel,Uyanga_ts,485391,119906,97.3,13.9
4,Д. Алтангэрэл,altancom7969,81889,9689,22.8,10.0
0,🟡 E.Odbayar Dēmos.mn ᠵᠣᠨ,Odbayar1206,19089,8877,4.4,12.1
3,Алтангэрэл Ц. %=🗳,aagiiah,10912,5079,34.9,0.9
1,САЛБАДАЙ,chinbat84,4021,1454,1.1,10.6


In [ ]:
full_neg=get_name(negative.head(), neg_user_stats)

In [ ]:
display_full(full_neg)

,name,screen_name,tweets,favorites,date
0,🟡 E.Odbayar Dēmos.mn ᠵᠣᠨ,Odbayar1206,Мөнхжин нарын залуусын хэргийг шүгэлддэг данх захиалж шоронд хийлгэсэн юм байна. Улс төрийн хэрэг гэдэг нь үнэн байна. Нууцаа ярих гээд хулгай дээрмээ ярив. https://t.co/dbLh5TcHkf,35,2023-02-20 02:57:57
1,САЛБАДАЙ,chinbat84,Шүгэлддэг данх \n• Зохиолтой\n• Тавилтай\n• Монтажтай\n• Цензуртэй\nНэвтрүүлэгт ороо ч нэмэр алга. Дандаа өөрийнхөө хулгайг ярих юм шууд эфирээр 5-6 сэтгүүлчдийн асуултад ээлжлэн хариулбал исгэрсээр байгаад дуусах байх даа. \n https://t.co/UQOAo2jzkr,35,2023-02-20 03:05:39
2,Uyanga Tsoggerel,Uyanga_ts,Энэ Данх одоо юун гонгиноон хийцийн бэ? Өөрийгөө шударга ёсны төлөө тэмцэгч болгож өрөвдүүлэх гээд бгаамуу?,23,2023-02-20 03:00:02
3,Алтангэрэл Ц. %=🗳,aagiiah,Айлд ороод хиам идсэн чинь тэр нь #ДАНХ-ыг долоож авсан мөнгөөрөө авсан хиам байвал яах уу ? Идмээргүй санагдчихлаа 😂,12,2023-02-20 01:06:50
4,Д. Алтангэрэл,altancom7969,Өчигдөр орой айл бүрийн цэнхэр дэлгэцээр хоосон данх баахан шүгэлдсэн гэв үү дээ? https://t.co/wEqQXOwiRb,11,2023-02-20 02:08:20


In [ ]:
print_screen_name(full_neg)

@Odbayar1206,
@chinbat84,
@Uyanga_ts,
@aagiiah,
@altancom7969,


In [ ]:
negative.to_csv(f'negative_{DATE}.csv',index=False)

In [ ]:
DATE="2023-02-20"
SEARCH_WORD="Л.Оюун-Эрдэнэ:"
tweets = get_tweets_csv(SEARCH_WORD, DATE)
tweets=tweets.sort_values("favorites", ascending=False)
print(f"Total of tweets: {tweets.shape[0]}, Total of favorites: {tweets.favorites.sum()}, a total of retweets: {tweets.retweets.sum()}")
tweets.head(10)

Total of tweets: 49, Total of favorites: 241, a total of retweets: 140


,id,user,tweets,date,location,favorites,retweets
48,1627465618072170496,Gerlee_smile,"Оюун-Эрдэнэ бол чадамжгүй ЕС, улс орон ямар байгаагаар ажлыг нь дүгнэдэг болохоос худлаа ярьснаар нь дүгнэхгүй л дээ.",2023-02-20 00:30:14,Mongolia,98,34
11,1627596348353818624,Batuka_UB,"Ерөнхий сайд Л.Оюун-Эрдэнэ өнөөдөр АН-ын дарга Л.Гантөмөрийг хүлээн авч уулзан амжилт хүслээ. \nУулзалтад УИХ дахь АН-ын бүлгийн дарга О.Цогтгэрэл, МАН-ын бүлгийн дарга Д.Тогтохсүрэн нар оролцож улс төр, эдийн засаг, парламентын засаглалыг хэрхэн бэхжүүлэх талаар санал солилцлоо🇲🇳 https://t.co/GejM6vrQ2f",2023-02-20 09:09:43,Улаанбаатар хот Монгол Улс .,45,26
21,1627552546142457856,Bolort,"Ерөнхий сайд, МАН-ын дарга Л.Оюун-Эрдэнэ өнөөдөр АН-ын даргаар сонгогдсон Л.Гантөмөрийг хүлээн авч уулзан амжилт хүслээ\n\nПарламентын засаглалыг бэхжүүлэх, авлигын эсрэг хийж буй Ерөнхий сайдын тэмцлийг дэмжихээ АН-ын дарга Л.Гантөмөр илэрхийлэв. https://t.co/5b0QqSJkDX https://t.co/N3wVVkPMj1",2023-02-20 06:15:40,Ulaanbaatar,22,13
20,1627553674821255168,miigaasbntv,"Ерөнхий сайд, МАН-ын дарга Л.Оюун-Эрдэнэ АН-ын даргаар сонгогдсон Л.Гантөмөрийг өнөөдөр хүлээн авч уулзан амжилт хүсчээ‼️🇲🇳 https://t.co/Cu78iHns4B",2023-02-20 06:20:09,,13,3
37,1627498759189049344,TewsMN,"Ерөнхий сайд Л.Оюун-Эрдэнэ: \n""Засгийн газрын эсрэг жагсаалыг IHC санхүүжүүлсэн. Мөн гадны санхүүжилттэй жагсаал болсон."" https://t.co/19Zv7LOgTw",2023-02-20 02:41:56,Mongolia,11,14
38,1627488895662280706,Siidaa09,ЕС Л.Оюун-Эрдэнэ: Боомт хаагдвал юу болдог вэ? Бид харлаа шүү дээ. Тэгвэл цаашид бензин тасарвал яах вэ? Эрчим хүчний хомсдолд орвол яах вэ? Бид гаднаас хамааралтай байгаа эдгээр хүчин зүйлүүдээ маш хурдан цэгцлэх ёстой гэж үзэж байна хэмээн онцоллоо. \n#mnb,2023-02-20 02:02:44,,10,6
46,1627474653273890817,Bayaraa0218,Л.Оюун-Эрдэнэ: Өнгөрсөн 30 жил бид бүх гоё үгээ хэлчихсэн. Харин одоо хийх л үлдсэн\n https://t.co/KQYgBfpSeK,2023-02-20 01:06:09,Ulaanbaatar.Mongolia,7,6
44,1627478571525578752,newsflowmn,Ерөнхий сайд юу хэлэв https://t.co/JdbDscXqbV \nЕрөнхий сайд Л.Оюун-Эрдэнэ МҮОНТ-д бүрэн хэмжээний ярилцлага өглөө. Түүний ярилцлагаас онцлох хэсгүүдийг хүргэж байна. \n@iToim_mn @OyunerdeneMN @Saruul_dul https://t.co/gVLzJnLqIm,2023-02-20 01:21:43,Mongolia,3,7
10,1627624741854138372,Ana1iz_mn,"Ерөнхий сайд Л.Оюун-Эрдэнэ АН-ын даргаар сонгогдсон Л.Гантөмөр, УИХ дахь АН-ын бүлгийн дарга О.Цогтгэрэл нарыг хүлээн авч уулзлаа. https://t.co/qKwlsuho5Q",2023-02-20 11:02:32,,3,3
6,1627645898518831104,ayvengoroe,"Сонгогчид-бид хулхидуулж ханахгүйм. Одоо МАН-ын худалч нарт итгээд сонговол 2028 оны зун хүртэл нар үзэхгүй. Эд нар 2016 оноос хойш 7 жил дангаараа засаглахдаа юу ч хийсэнгүй, хэн нэгнийг ч хийлгэсэнгүй. Цаашдаа ч хийхгүйм байна. Оюун-Эрдэнэ хулчгар, засгаа наймаалцаад л дууслаа.",2023-02-20 12:26:37,"Ulaanbaatar, Mongolia",3,1


In [ ]:
positive=tweets[tweets.user!="Gerlee_smile"]
pos_user_stats=get_stats_with_name(positive.head())
pos_user_stats.sort_values("followers_count", ascending=False)
full_pos=get_name(positive.head(), neg_user_stats)
display_full(full_pos)

,name,screen_name,tweets,favorites,date
0,Батука,Batuka_UB,"Ерөнхий сайд Л.Оюун-Эрдэнэ өнөөдөр АН-ын дарга Л.Гантөмөрийг хүлээн авч уулзан амжилт хүслээ. \nУулзалтад УИХ дахь АН-ын бүлгийн дарга О.Цогтгэрэл, МАН-ын бүлгийн дарга Д.Тогтохсүрэн нар оролцож улс төр, эдийн засаг, парламентын засаглалыг хэрхэн бэхжүүлэх талаар санал солилцлоо🇲🇳 https://t.co/GejM6vrQ2f",45,2023-02-20 09:09:43
1,Bolortuya Chuluunbaatar,Bolort,"Ерөнхий сайд, МАН-ын дарга Л.Оюун-Эрдэнэ өнөөдөр АН-ын даргаар сонгогдсон Л.Гантөмөрийг хүлээн авч уулзан амжилт хүслээ\n\nПарламентын засаглалыг бэхжүүлэх, авлигын эсрэг хийж буй Ерөнхий сайдын тэмцлийг дэмжихээ АН-ын дарга Л.Гантөмөр илэрхийлэв. https://t.co/5b0QqSJkDX https://t.co/N3wVVkPMj1",22,2023-02-20 06:15:40
2,Мягмартогтох Далай,miigaasbntv,"Ерөнхий сайд, МАН-ын дарга Л.Оюун-Эрдэнэ АН-ын даргаар сонгогдсон Л.Гантөмөрийг өнөөдөр хүлээн авч уулзан амжилт хүсчээ‼️🇲🇳 https://t.co/Cu78iHns4B",13,2023-02-20 06:20:09
3,Tews,TewsMN,"Ерөнхий сайд Л.Оюун-Эрдэнэ: \n""Засгийн газрын эсрэг жагсаалыг IHC санхүүжүүлсэн. Мөн гадны санхүүжилттэй жагсаал болсон."" https://t.co/19Zv7LOgTw",11,2023-02-20 02:41:56
4,𝗦𝗶𝗶𝗱𝗮𝗮 𝗕𝗮𝘁𝗯𝗮𝘆𝗮𝗿,Siidaa09,ЕС Л.Оюун-Эрдэнэ: Боомт хаагдвал юу болдог вэ? Бид харлаа шүү дээ. Тэгвэл цаашид бензин тасарвал яах вэ? Эрчим хүчний хомсдолд орвол яах вэ? Бид гаднаас хамааралтай байгаа эдгээр хүчин зүйлүүдээ маш хурдан цэгцлэх ёстой гэж үзэж байна хэмээн онцоллоо. \n#mnb,10,2023-02-20 02:02:44


In [ ]:
neg_user_stats

,name,screen_name,statuses_count,followers_count,average_tweets_per_day,account_age_year
0,Батука,Batuka_UB,157488,14036,46.1,9.5
1,Bolortuya Chuluunbaatar,Bolort,31444,52791,6.4,13.7
2,Мягмартогтох Далай,miigaasbntv,21393,17967,13.1,4.5
3,Tews,TewsMN,127,165,0.4,1.0
4,𝗦𝗶𝗶𝗱𝗮𝗮 𝗕𝗮𝘁𝗯𝗮𝘆𝗮𝗿,Siidaa09,10838,2337,27.8,1.1


In [ ]:
print_screen_name(full_pos)

@Batuka_UB,
@Bolort,
@miigaasbntv,
@TewsMN,
@Siidaa09,


In [ ]:
positive.to_csv(f'positive_{DATE}.csv',index=False)

### Sort df by favorites

In [ ]:
key_word="чухал"
DATE="2023-02-01"
df=get_tweets_csv(key_word, DATE)
df=df.sort_values("favorites", ascending=False)
df.head(6)
df.to_csv(f'{key_word}.csv',index=False)
df1=pd.read_csv(f'{key_word}.csv')
df1=df1[["user","tweets","favorites", "date"]]
df1.head(6)
df2=df1.head(100)
df2

,user,tweets,favorites,date
0,Siidaa09,"Компанийн хувьд хамгийн чухал нь хүн.Техник, т...",391,2023-02-13 13:02:38
1,dbtseegii,Энэ ёстой зөв чухал асуудал хөнджээ. Олуулаа т...,282,2023-02-17 06:16:19
2,shaba0802,"Яг manipulation, insider trade хийсэн үү гэдэг...",186,2023-02-14 07:10:01
3,Batkhuu_lw,Бидний олонх нь боловсрол мэдлэг чухал биш гол...,132,2023-02-11 01:14:04
4,batzayased,@Gvnjidmaa Уучлаарай. Ийм үед итгэл найдвар чу...,112,2023-02-10 07:09:20
...,...,...,...,...
95,Ariunq,Яг иймэрхүү аргаар дамжуулж хүүхдүүдэд хүнлэг ...,11,2023-02-12 03:31:07
96,argalchin_ovogt,@Tsenguun99999 өө тэгээд өвөр дээр нь өссөн л ...,11,2023-02-16 13:50:59
97,NaigalBatkhuyag,@NTVmn @sambuu01201 Аливаа улс орны хувьд хамг...,11,2023-02-14 10:55:33
98,olonr_R,@EBathul Хэзээ авсан нь чухал биш. Яаж баригдс...,11,2023-02-12 13:35:57


In [ ]:
df=get_tweets_csv("Л.Оюун-Эрдэнэ", "2023-02-20")
df=df.sort_values("favorites", ascending=False)
df

,id,user,tweets,date,location,favorites,retweets


In [ ]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:803, Total favorites:5424


### Sentiment analysis using Google NLP API

In [ ]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
def do_sentiment(quote):
  try:
    response = lservice.documents().analyzeSentiment(
    body={
      'document': {
          'type': 'PLAIN_TEXT',
          'content': quote
      }
    }).execute()
    polarity = response['documentSentiment']['polarity']
    # magnitude = response['documentSentiment']['magnitude']
    return polarity
  except:
    polarity=0
    return polarity

### Before calling NLP API, we need to translate Mongolian tweets to English
### using google translate API

In [ ]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='mn', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> энэ үнэхээр амархан уу?
amazing technology -> гайхалтай технологи
wow -> хөөх


In [ ]:
def translate_it(inputs):
  outputs = service.translations().list(source='mn', target='en', q=inputs).execute()
  return outputs['translations'][0]['translatedText']

### Run NLP API over each tweets to find polarity

In [ ]:
df2['english']=df2.tweets.map(lambda x:translate_it(x))
df2['polarity']=df2.english.map(lambda x:do_sentiment(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2=df2[['user','polarity','tweets','favorites']]
df2

,user,polarity,tweets,favorites
0,DanzanBolormaa,0.4,За Энхбат Хүрэлсүх 2ын ярилцлага үзлээ. Боловс...,274
1,g_ganbat,-0.8,АНУ-ын Мянганы сорилын сангийн УБ-ын усан ханг...,242
2,Mandolinchin,0.9,"Нэр дэвшиж буй 3 эрхэм Энхбат ах, найз Хүрэлсү...",183
3,tvdaria,-0.1,С.Эрдэнэ 2007онд Солонгост ажиллаж бсан иргэди...,162
4,batengineer,-1.0,"Хүрэлсүх, Эрдэнэ нарын сурталчилгаа интернет о...",155
...,...,...,...,...
95,amarsanaa,-0.1,У.Хүрэлсүх: Манай Засгийн газар өрийн удирдлаг...,10
96,Burmaa87,-1.0,МАНай Хүрэлсүх би бол эгэл ардын төлөөлөл гэсэ...,10
97,sakuna_mn,1.0,У.Хүрэлсүх: Аав ээжийгээ амьд сэрүүн байхад нь...,10
98,ariunq,-0.4,"Хүрэлсүх аав ээжийнхээ ачийг хариулсан, цуйван...",10


In [ ]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:951, Total favorites:6243


In [ ]:
# table_id='tweets.enkhbat'
# project_id = 'twittersheet-275317'
# pandas_gbq.to_gbq(df, table_id, project_id=project_id ,if_exists='replace')

1it [00:02,  2.77s/it]


### Visualization

In [ ]:
!ls

ananbuzar.csv	 enkhbat_v2.csv  khurelsukh.csv		     tweeters_v2.csv
anandbuzar.csv	 erdene.csv	 mongolian_top_tweeters.csv  tweets.ipynb
an.csv		 fiverr.csv	 samarsaihan.csv
battulga_v2.csv  hook.csv	 teneg_bandi.csv
enkhbat.csv	 hook_tweet.csv  top_active_tweeters.csv


In [ ]:
df_k=pd.read_csv('khurelsukh.csv')

In [ ]:
df_e=pd.read_csv('enkhbat_v2.csv')

In [ ]:
df_e

,id,user,tweets,date,location,favorites,retweets
0,1394216127404802050,enkhbat,"Уралын өвөл. Оюутан цаг. Энхбат, Оюунаа хоёр....",2021-05-17 09:00:02,Ulaanbaatar,846,110
1,1394245887501619204,enkhbat,"Инженер Энхбат. 1990. Компьютер, сүлжээний тух...",2021-05-17 10:58:18,Ulaanbaatar,676,140
2,1394862654108278787,bambgar19,"Э.Энхбат начин ухаан орж,амь нас нь үхлээс хол...",2021-05-19 03:49:06,Mongolia,448,90
3,1393810304996364289,batengineer,Нэр дэвшигч Энхбат гуайг муулах өнцөг олох гэж...,2021-05-16 06:07:27,Mongolia,391,164
4,1392845222057418759,Sanchir_philo,"Энхбат гуайг дэмжиж буй хүмүүс сэхээтэн,оюунла...",2021-05-13 14:12:33,"Wien, Österreich",378,124
...,...,...,...,...,...,...,...
1344,1394198984135254017,tuvshee1128,@Mandukhai_ Тийм шүү.Энхбат интернэт оруулж ир...,2021-05-17 07:51:55,NaN,0,1
1345,1394197629177720834,anba_mk,Хаха өөр нэг залуу хийх л байсан. Энхбат азтай...,2021-05-17 07:46:32,NaN,0,1
1346,1394195462836539393,tulgaa68,@Mandukhai_ Өөр хэн нэгэн Энхбат бх л бсан даа...,2021-05-17 07:37:55,Mongolia,0,0
1347,1394193792094322689,idevkhtei_irgen,Д. Энхбат болон У.Хүрэлсүх гэдэг 2 хүний тухай...,2021-05-17 07:31:17,Улаанбаатар. Монгол Улс,0,0


In [ ]:
df_e=df_e.head(800)

In [ ]:
# df_e['english']=df_e.tweets.map(lambda x:translate_it(x))
df_e['polarity']=df_e.english.map(lambda x:do_sentiment(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def labeling(x):
  if x >0.3:
    return 'positive'
  elif x<-0.3:
    return 'negative'
  else:
    return 'neutral'

### U.Khurelsukh

In [ ]:

df_k['Expression Label'] = df_k.polarity.map(lambda x: labeling(x) )
df_k.tail()

,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
798,1393908343693926402,Narantuya_P,Гадуур хүмүүс Хүрэлсүхээ сонгоноо л гэж байна....,2021-05-16 12:37:01,Mongolia,0,1,Outsiders say they will choose Khurelsukhe. On...,0.2,neutral
799,1395261188313804800,BaagiiJr,"Хүрэлсүх дайныг, танкыг, бууг зохиосон хүн шүү...",2021-05-20 06:12:44,NaN,0,0,"He invented the war of touch, tanks and guns",1.0,positive
800,1395259293209808897,Bilbook0815,Монгол улсад хууль дүрэм байна уу?Суртчилгаа э...,2021-05-20 06:05:12,NaN,0,0,Is there any law in Mongolia? U.Khurelsukh sta...,-0.5,negative
801,1393916458548166664,Amarsanaa006,@Aldar51 @ariunq Хүрэлсүх ганцаараа биш шүү бу...,2021-05-16 13:09:16,Mongolia,0,0,"@ Aldar51 @ariunq Khurelsukh is not alone, eve...",-1.0,negative
802,1394295007725584384,AnjinZaVoid,@MunkhchuluunS Хүрэлсүх байгаагүй бол сайхаг б...,2021-05-17 14:13:29,ub,0,0,@MunkhchuluunS It wouldn&#39;t be nice without...,1.0,positive


In [ ]:
new1 = df_k.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 802 tweets about the president candidate U.Khurelsukh",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_kh.to_csv('khurelsukh_with_sentiment.csv', index=False)

### D.Enkhbat

In [ ]:
df_e['Expression Label'] = df_e.polarity.map(lambda x: labeling(x) )
df_e.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
795,1393052334033235969,Gereltod_Tsog,Энхбат гэдэг үгийг mute хийчихлээ 😆,2021-05-14 03:55:32,Mongolei,1,0,The word Enkhbat has been mute 😆,1.0,positive
796,1394165285347856388,Batusolu,Энхбат орж ирсэнээр Хүрэлсүх ерөнхийлөгч болох...,2021-05-17 05:38:01,хот,1,1,"With Enkhbat coming in, the chances of Khurels...",0.5,positive
797,1394909744234594308,chaimkaaaa,@bo_ulzii Аанхаа энхбат начин би сая эхнэртэй ...,2021-05-19 06:56:13,Mongolia,1,0,"@bo_ulzii Well, I just got in touch with his w...",0.1,neutral
798,1394508797205491713,Nyambaizula5,@Babrosk Энхбат байгаагүй бол интернэт байхгүй...,2021-05-18 04:23:00,"Budapest, Hungary",1,0,"@Babrosk He said that without Enkhbat, there w...",-1.0,negative
799,1393560958199549952,batjargaln,Хүрэлсүх: Би чинь муу ч гэсэн хурандаа хүн ш т...,2021-05-15 13:36:38,Somewhere in the Internet,1,0,Touch: I&#39;m a bad colonel. Erdene: You can ...,-0.4,negative


In [ ]:
new1 = df_e.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 800 tweets about the president candidate D.Enkhbat",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df=df_e.sort_values('date')
df=df[['Expression Label','tweets']]
df.to_csv('enkhbat_sent_analysis.csv', index=False)

In [ ]:
df_e.to_csv('enkhbat_with_sentiment.csv', index=False)

### S.Erdene

In [ ]:
df_er=pd.read_csv('erdene.csv')
df_er=df_er.head(800)

In [ ]:
# df_er['english']=df_er.tweets.map(lambda x:translate_it(x))
df_er['polarity']=df_er.english.map(lambda x:do_sentiment(x))

In [ ]:
df_er['Expression Label'] = df_er.polarity.map(lambda x: labeling(x) )
df_er.tail()

,id,user,tweets,date,location,favorites,retweets,english,polarity,Expression Label
795,1394465909578756103,timemn,Л.Оюун-Эрдэнэ: Дэлхийд гарч болох соёлын бүх к...,2021-05-18 01:32:35,Mongolia,1,0,L.Oyun-Erdene: Government will support all cul...,1.0,positive
796,1394522457403367426,odmandahk0106,Би эрдэнэ мэт ганц саналаа С.Эрдэнэд өгнө гэж ...,2021-05-18 05:17:17,"Hollands Kroon, Nederland",1,0,I decided to give my only vote to S.Erdene.,-1.0,negative
797,1394466420357046287,erdembileg2010,@AngaragDarigang Чам шиг янханд Эрдэнэ ямар га...,2021-05-18 01:34:37,NaN,1,1,"@AngaragDarigang Erdene, what happened to a pr...",-1.0,negative
798,1394522197020999681,aagii896,@myagmardorj_ Тэгээд л нөгөө хэд нь эрдэнэ гээ...,2021-05-18 05:16:15,Mongolia,1,0,@myagmardorj_ So how many of them are begging ...,-1.0,negative
799,1395027396969918469,tsdulmaa1,Зөв санаа👍👍Л.Оюун-Эрдэнэ: Төгсөөд хөдөө очиж а...,2021-05-19 14:43:44,NaN,1,0,Right idea👍👍L.Oyun-Erdene: If you go to work i...,1.0,positive


In [ ]:
df=df_er.sort_values('date')
df=df[['Expression Label','tweets']]
df.to_csv('s_erdene_sent_analysis.csv', index=False)

In [ ]:
new1 = df_er.groupby('Expression Label').count()
x = list(new1['polarity'])
y = list(new1.index)
tuple_list = list(zip(x,y))

df = pd.DataFrame(tuple_list, columns=['x','y'])

df['color'] = 'blue'
df['color'][1] = 'red'
df['color'][2] = 'green'

import plotly.graph_objects as go
fig = go.Figure(go.Bar(x=df['x'],
                y=df['y'],
                orientation ='h',
                marker={'color': df['color']}))
fig.update_layout(
    title={
        'text': "The latest 800 tweets about the president candidate S.Erdene",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

